In [57]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
import json    

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# !conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Define foursquare Credentials and Version

In [58]:
CLIENT_ID = 'IZ3OANXNNHAGM2NO1FBTZ4WWGRLJPQORTA1EC4LEY3GTWJGO' # your Foursquare ID
CLIENT_SECRET = 'A3DOWBNWOGPZKSVYP04IY2UE1FSFBQNIDSKGDLRYG5XVQAD1' # your Foursquare Secret
VERSION = '20200604'
LIMIT = 100
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: IZ3OANXNNHAGM2NO1FBTZ4WWGRLJPQORTA1EC4LEY3GTWJGO
CLIENT_SECRET:A3DOWBNWOGPZKSVYP04IY2UE1FSFBQNIDSKGDLRYG5XVQAD1


We're looking in the middle of Manhatten

In [59]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7896239 -73.9598939


Search for a specific venue category

In [60]:
search_query = 'Jazz'
# radius = 2000
print(search_query + ' .... OK!')

Jazz .... OK!


define and submit the url...

In [61]:
# url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    latitude, 
    longitude, 
    VERSION, 
    search_query, 
#     radius, 
    LIMIT
    )

results = requests.get(url).json()

In [62]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# # tranform venues into a dataframe
  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
# filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
# filtered_columns = ['name', 'categories', 'lat', 'lng']
# nearby_venues = nearby_venues.loc[:, filtered_columns]

# # filter the category for each row
# nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

# # clean columns and keep everything after the full spot
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-62-5b232762ffdc>:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,id,name,categories,referralId,hasPerk,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id,neighborhood
0,3fd66200f964a5205ee81ee3,Jazz on the Park Hostel,"[{'id': '4bf58dd8d48988d1ee931735', 'name': 'H...",v-1591625846,False,36 W 106th St,Central Park West,40.798309,-73.960995,"[{'label': 'display', 'lat': 40.79830920182899...",971,10025,US,New York,NY,United States,"[36 W 106th St (Central Park West), New York, ...",NaN,NaN
1,4d377322dd896a31ac8a2921,Jazz on Amsterdam Ave Hostel,"[{'id': '4bf58dd8d48988d1ee931735', 'name': 'H...",v-1591625846,False,"201 West 87th St, New York",NaN,40.788547,-73.974901,"[{'label': 'display', 'lat': 40.78854725887858...",1270,10024,US,New York,NY,United States,"[201 West 87th St, New York, New York, NY 1002...",NaN,NaN
2,4b980f59f964a520ac2935e3,Jazz at Lincoln Center,"[{'id': '4bf58dd8d48988d1e7931735', 'name': 'J...",v-1591625846,False,10 Columbus Cir,at W 60th St,40.768224,-73.982792,"[{'label': 'display', 'lat': 40.76822422081554...",3066,10023,US,New York,NY,United States,"[10 Columbus Cir (at W 60th St), New York, NY ...",NaN,NaN
3,4c47a87496abd13a66be7101,Smoke Jazz & Supper Club,"[{'id': '4bf58dd8d48988d1e5931735', 'name': 'M...",v-1591625846,False,2751 Broadway,btwn W 105th & W 106th St,40.801075,-73.967866,"[{'label': 'display', 'lat': 40.80107489000833...",1440,10025,US,New York,NY,United States,"[2751 Broadway (btwn W 105th & W 106th St), Ne...",48325453,NaN
4,4beec9012c082d7f76da3042,Jazz Museum of Harlem,"[{'id': '4bf58dd8d48988d181941735', 'name': 'M...",v-1591625846,False,104 E 126th St,at Park Ave,40.805447,-73.938344,"[{'label': 'display', 'lat': 40.8054469157015,...",2529,10035,US,New York,NY,United States,"[104 E 126th St (at Park Ave), New York, NY 10...",NaN,NaN


Download the newyork dataset

In [63]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [64]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [65]:
neighborhoods_data = newyork_data['features']

Transform the data into a pandas dataframe

In [66]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [67]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

#### Create a map of New York with neighborhoods superimposed on top.

In [68]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [71]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, label in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

explore Neighborhoods of Manhattan

In [72]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [60]:
venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )


Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [74]:
# print(manhattan_venues.shape)
# manhattan_venues.head()

nearby_venues.shape
nearby_venues.head()

,id,name,categories,referralId,hasPerk,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id,neighborhood
0,3fd66200f964a5205ee81ee3,Jazz on the Park Hostel,"[{'id': '4bf58dd8d48988d1ee931735', 'name': 'H...",v-1591625846,False,36 W 106th St,Central Park West,40.798309,-73.960995,"[{'label': 'display', 'lat': 40.79830920182899...",971,10025,US,New York,NY,United States,"[36 W 106th St (Central Park West), New York, ...",NaN,NaN
1,4d377322dd896a31ac8a2921,Jazz on Amsterdam Ave Hostel,"[{'id': '4bf58dd8d48988d1ee931735', 'name': 'H...",v-1591625846,False,"201 West 87th St, New York",NaN,40.788547,-73.974901,"[{'label': 'display', 'lat': 40.78854725887858...",1270,10024,US,New York,NY,United States,"[201 West 87th St, New York, New York, NY 1002...",NaN,NaN
2,4b980f59f964a520ac2935e3,Jazz at Lincoln Center,"[{'id': '4bf58dd8d48988d1e7931735', 'name': 'J...",v-1591625846,False,10 Columbus Cir,at W 60th St,40.768224,-73.982792,"[{'label': 'display', 'lat': 40.76822422081554...",3066,10023,US,New York,NY,United States,"[10 Columbus Cir (at W 60th St), New York, NY ...",NaN,NaN
3,4c47a87496abd13a66be7101,Smoke Jazz & Supper Club,"[{'id': '4bf58dd8d48988d1e5931735', 'name': 'M...",v-1591625846,False,2751 Broadway,btwn W 105th & W 106th St,40.801075,-73.967866,"[{'label': 'display', 'lat': 40.80107489000833...",1440,10025,US,New York,NY,United States,"[2751 Broadway (btwn W 105th & W 106th St), Ne...",48325453,NaN
4,4beec9012c082d7f76da3042,Jazz Museum of Harlem,"[{'id': '4bf58dd8d48988d181941735', 'name': 'M...",v-1591625846,False,104 E 126th St,at Park Ave,40.805447,-73.938344,"[{'label': 'display', 'lat': 40.8054469157015,...",2529,10035,US,New York,NY,United States,"[104 E 126th St (at Park Ave), New York, NY 10...",NaN,NaN


In [76]:
nearby_venues.groupby('neighborhood').count()

,id,name,categories,referralId,hasPerk,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
neighborhood,,,,,,,,,,,,,,,,,,
Koreatown,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0


how many unique categories

In [63]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 331 uniques categories.


In [105]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [106]:
manhattan_onehot.shape

(3113, 332)

In [107]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Battery Park City,0.000000,0.00,0.00,0.000000,0.015385,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.015385,0.00000,0.000000,0.000000
1,Carnegie Hill,0.000000,0.00,0.00,0.000000,0.011905,0.00,0.000000,0.000000,0.011905,...,0.00,0.023810,0.000000,0.000000,0.000000,0.011905,0.035714,0.00000,0.000000,0.035714
2,Central Harlem,0.000000,0.00,0.00,0.065217,0.043478,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
3,Chelsea,0.000000,0.00,0.00,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.00000,0.010000,0.000000
4,Chinatown,0.000000,0.00,0.00,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,...,0.00,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.010000
5,Civic Center,0.000000,0.00,0.00,0.000000,0.040000,0.01,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.010000,0.010000,0.010000,0.01000,0.000000,0.030000
6,Clinton,0.000000,0.00,0.00,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.040000,0.00000,0.000000,0.000000
7,East Harlem,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
8,East Village,0.000000,0.00,0.00,0.000000,0.010000,0.00,0.000000,0.010000,0.010000,...,0.00,0.020000,0.000000,0.000000,0.000000,0.030000,0.010000,0.00000,0.000000,0.000000
9,Financial District,0.000000,0.00,0.00,0.000000,0.050000,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.00000,0.010000,0.010000


In [108]:
manhattan_grouped.shape

(40, 332)

In [109]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
           venue  freq
0           Park  0.12
1          Hotel  0.06
2    Coffee Shop  0.05
3  Memorial Site  0.05
4            Gym  0.05


----Carnegie Hill----
                 venue  freq
0          Coffee Shop  0.10
1                 Café  0.05
2          Pizza Place  0.05
3          Yoga Studio  0.04
4  Japanese Restaurant  0.04


----Central Harlem----
                  venue  freq
0    African Restaurant  0.07
1  Gym / Fitness Center  0.04
2    Seafood Restaurant  0.04
3                   Bar  0.04
4    Chinese Restaurant  0.04


----Chelsea----
            venue  freq
0     Coffee Shop  0.08
1     Art Gallery  0.08
2          Bakery  0.04
3  Ice Cream Shop  0.04
4            Café  0.03


----Chinatown----
                venue  freq
0  Chinese Restaurant  0.07
1              Bakery  0.05
2        Cocktail Bar  0.04
3        Optical Shop  0.03
4                 Bar  0.03


----Civic Center----
                 venue  freq
0          Coffee Shop  0.08
1 

In [110]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [111]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Hotel,Gym,Coffee Shop,Memorial Site,Food Court,Plaza,Playground,Boat or Ferry,Beer Garden
1,Carnegie Hill,Coffee Shop,Café,Pizza Place,Japanese Restaurant,Gym / Fitness Center,Gym,Yoga Studio,Bookstore,Wine Shop,Cocktail Bar
2,Central Harlem,African Restaurant,Gym / Fitness Center,Bar,Chinese Restaurant,American Restaurant,French Restaurant,Seafood Restaurant,Market,Gym,Library
3,Chelsea,Art Gallery,Coffee Shop,Bakery,Ice Cream Shop,Café,American Restaurant,Italian Restaurant,Market,Seafood Restaurant,Cocktail Bar
4,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,American Restaurant,Optical Shop,Bar,Spa,Vietnamese Restaurant,Bubble Tea Shop,Ice Cream Shop


In [113]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [114]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 0, 1, 0, 1, 1, 3, 0, 1], dtype=int32)

In [115]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,4,Sandwich Place,Coffee Shop,Gym,Bank,Supplement Shop,Steakhouse,Miscellaneous Shop,Shopping Mall,Seafood Restaurant,Yoga Studio
1,Manhattan,Chinatown,40.715618,-73.994279,0,Chinese Restaurant,Bakery,Cocktail Bar,American Restaurant,Optical Shop,Bar,Spa,Vietnamese Restaurant,Bubble Tea Shop,Ice Cream Shop
2,Manhattan,Washington Heights,40.851903,-73.936900,3,Café,Bakery,Mobile Phone Shop,Spanish Restaurant,Mexican Restaurant,Coffee Shop,New American Restaurant,Tapas Restaurant,Latin American Restaurant,Park
3,Manhattan,Inwood,40.867684,-73.921210,3,Mexican Restaurant,Café,Lounge,Restaurant,Bakery,Caribbean Restaurant,Spanish Restaurant,Frozen Yogurt Shop,Chinese Restaurant,American Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,3,Pizza Place,Coffee Shop,Deli / Bodega,Mexican Restaurant,Café,Cocktail Bar,Indian Restaurant,Sushi Restaurant,Park,Yoga Studio


In [118]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters